1. Read json file
2. Set up the sampler through batchrunner
3. generate the input file
4. save it.

In [59]:
# Import relevant libraries
import os
from ModularCirc import BatchRunner
import numpy as np
import pandas as pd
from ModularCirc.Models.NaghaviModel import NaghaviModel, NaghaviModelParameters, TEMPLATE_TIME_SETUP_DICT

In [60]:
os.getcwd()

'/Users/syonis/project/comparative-gsa/sandbox'

In [61]:
# Define variables
param_path = "./parameters_01.json"

n_samples = 128 # Powers of 2

simulation_out_path = f'outputs/simulations/output_{n_samples}_samples/'

In [62]:
# Set up the batch runner
br = BatchRunner('Sobol', 0) # why are we using 'Sobol' here? 
br.setup_sampler(param_path)
br.sample(n_samples)

In [63]:
map_ = {
    'lv.t_tr' : ['lv.t_tr',],
    'la.t_tr' : ['la.t_tr',],
    'la.delay' : ['la.delay',],
    'lv.tau' : ['lv.tau',],
    'la.tau' : ['la.tau',],
    'lv.t_max' : ['lv.t_max',],
    'la.t_max' : ['la.t_max',],
}
br.map_sample_timings(
    ref_time=1000., # double check if 1000 or 1
    map=map_
    )

In [64]:
br.map_vessel_volume()

In [65]:
br.samples

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,la.t_tr,la.tau,la.t_max,lv.delay,lv.t_tr,lv.tau,lv.t_max,ao.v,art.v,ven.v
0,324.140512,0.429410,970.557289,2.593650,5.759509,141.391649,5.535208,3.645771,0.302181,0.614511,...,103.265426,11.473936,68.843618,0,192.762129,11.473936,128.508086,11.183394,67.548099,3682.353190
1,213.211374,0.168363,1134.582900,3.092750,11.457833,81.051107,6.337772,5.927350,0.476733,0.449234,...,264.970053,29.441117,176.646702,0,494.610765,29.441117,329.740510,12.893425,236.845967,6206.977557
2,128.027561,0.373035,664.791603,1.807716,9.775524,110.934879,3.831676,4.690519,0.657067,0.244713,...,173.562758,19.284751,115.708506,0,323.983815,19.284751,215.989210,14.695447,71.213695,4370.200691
3,287.120769,0.262023,1448.888220,3.761496,8.444797,167.239972,7.529157,2.905174,0.338195,0.522401,...,132.781531,14.753503,88.521021,0,247.858858,14.753503,165.239239,8.718682,125.161948,5564.827928
4,261.317508,0.322630,1342.111306,4.190009,7.184780,154.969280,3.238999,4.237066,0.583734,0.372392,...,146.342358,16.260262,97.561572,0,273.172401,16.260262,182.114934,10.757573,139.708922,5167.193956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,260.373346,0.247353,999.093670,1.799288,6.390333,191.825352,4.329372,4.456349,0.483956,0.664860,...,253.575478,28.175053,169.050319,0,473.340893,28.175053,315.560595,6.882423,50.064029,5337.415680
124,286.191297,0.337256,683.573974,2.214709,4.840048,146.232722,8.711855,2.749402,0.245722,0.226446,...,240.013102,26.668122,160.008735,0,448.024458,26.668122,298.682972,11.363779,74.624331,4927.291974
125,130.863174,0.297729,1430.284752,4.198426,12.922972,73.395301,3.421036,5.030031,0.530185,0.511224,...,78.988206,8.776467,52.658804,0,147.444651,8.776467,98.296434,18.178675,256.346526,4481.353607
126,210.390852,0.393705,914.353541,2.952212,11.124808,101.179480,7.427087,5.843199,0.604011,0.601960,...,157.046341,17.449593,104.697561,0,293.153170,17.449593,195.435447,21.964826,164.704082,5644.808958


In [66]:
# Save the samples to a CSV file
output_path = "./samples.csv"

br.samples.to_csv(output_path, index=False)

In [67]:
br.setup_model(model=NaghaviModel, po=NaghaviModelParameters,
                   time_setup=TEMPLATE_TIME_SETUP_DICT)

In [68]:
os.makedirs(simulation_out_path, exist_ok=True)

In [69]:
simulations = br.run_batch(
            n_jobs=10,
            output_path=simulation_out_path,
        )



























100%|██████████| 128/128 [00:11<00:00, 10.92it/s]


In [74]:
import pandas as pd

headers = [
    'v_ao', 'v_art', 'v_ven', 'v_la', 'v_lv', 'p_lv', 'q_av', 'p_ao', 'p_art',
    'q_ao', 'p_ven', 'q_art', 'p_la', 'q_ven', 'q_mv', 'T'
]

summary_rows = []

for i_sim in range(len(simulations)):
    sim_df = simulations[i_sim]
    
    # Skip if not a DataFrame
    if not isinstance(sim_df, pd.DataFrame):
        print(f"Simulation {i_sim} is not a DataFrame, skipping.")
        continue

    row = {}
    for h in headers:
        row[f"{h}_mean"] = sim_df[h].mean()
        row[f"{h}_max"] = sim_df[h].max()
    summary_rows.append(row)
    print(f"Simulation {i_sim} done.")

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv('simulation_summary.csv', index=False)
print("Saved as simulation_summary.csv")

Simulation 0 done.
Simulation 1 done.
Simulation 2 done.
Simulation 3 done.
Simulation 4 done.
Simulation 5 done.
Simulation 6 done.
Simulation 7 done.
Simulation 8 done.
Simulation 9 done.
Simulation 10 done.
Simulation 11 done.
Simulation 12 done.
Simulation 13 done.
Simulation 14 done.
Simulation 15 done.
Simulation 16 done.
Simulation 17 done.
Simulation 18 done.
Simulation 19 done.
Simulation 20 done.
Simulation 21 done.
Simulation 22 done.
Simulation 23 done.
Simulation 24 done.
Simulation 25 done.
Simulation 26 done.
Simulation 27 done.
Simulation 28 done.
Simulation 29 done.
Simulation 30 done.
Simulation 31 is not a DataFrame, skipping.
Simulation 32 done.
Simulation 33 done.
Simulation 34 done.
Simulation 35 done.
Simulation 36 done.
Simulation 37 done.
Simulation 38 done.
Simulation 39 done.
Simulation 40 done.
Simulation 41 done.
Simulation 42 done.
Simulation 43 done.
Simulation 44 done.
Simulation 45 done.
Simulation 46 done.
Simulation 47 done.
Simulation 48 done.
Simula

In [ ]:
# Create the columns similar to Fay

headers = list(range(100)) + ['CO', 'mVa', 'EF', 'dPAP', 'sPAP', 'mPAP'] 
# This would change to ['mean_v', 'max_v', 'mean_q', 'max_q', 'mean_p', 'max_p']  or something similar. 
# Also the 100 would stay as 1000?

In [ ]:
# Assuming we dont resample, begin calculating mean/max v,q,p which will be the headers of the columns
# Since there are multiple different components for each parameter would it be:

